+ [위키독스참고](https://wikidocs.net/159468)


## [1] 경로설정

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Team_limemul/data_team/data

/content/drive/.shortcut-targets-by-id/1GZLytEEzicsdhwVm5wsQnDjfTXtcdk6h/Team_limemul/data_team/data


## [2] 데이터 로드

In [3]:
! pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from tqdm import tqdm
import glob, json, re
import pandas as pd

filenames = glob.glob('./news_2020_csv/*.csv')


In [10]:
df0 = pd.read_csv(filenames[0])
print(df0.shape)
print(df0.columns)

(18076, 4)
Index(['date', 'title', 'topic', 'description'], dtype='object')


## [3] keyBert

### (1) 설치

In [ ]:
!pip install sentence_transformers
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.5 MB/s 
     |████████████████████████████████| 5.3 MB 50.7 MB/s 
     |████████████████████████████████| 1.3 MB 57.0 MB/s 
     |████████████████████████████████| 163 kB 64.8 MB/s 
     |████████████████████████████████| 7.6 MB 50.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=fa8aab1d17087f21e39c71fd67f5ad358b5452bba3317ce48a32d0a5bbd26faf
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 154 kB/s 
     |████████████████████████████████| 453 kB 79.5 MB/s 


### (2) 패키지로드

In [ ]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

### (3) 명사추출 - Okt

In [ ]:
okt = Okt()
doc = df0.loc[1,'description']
# doc = eco_docs
tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)



n_gram_range = (1, 2)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

품사 태깅 10개만 출력 : [('저축은행', 'Noun'), ('업계', 'Noun'), ('가', 'Josa'), ('올해', 'Noun'), ('성장', 'Noun'), ('을', 'Josa'), ('위해', 'Noun'), ('‘', 'Foreign'), ('디지털', 'Noun'), ('인재', 'Noun')]
명사 추출 : 저축은행 업계 올해 성장 위해 디지털 인재 사활 웰컴 저축은행 웰컴 디지털 뱅크 웰뱅 층 저축은행 가망 고객 수 점 확인 금융 하나 금융 앱 모든 시중 은행 업무 오픈 뱅킹 시행 저축은행 역시 디지털 투자 수 환경 조성 것 금융감독원 금융 통계 정보 시스템 지난해 기준 저축은행 개사 총 임 직원 수 합계 명 지난해 기간 명 견줘 증가 성장 추세 지난해 명 변화 여명 결과 것 구체 웰컴 저축은행 명 명 저축은행 명 명 페퍼 저축은행 명 명 비롯 저축은행 명 명 한국 투자 저축은행 명 명 저축은행 명 명 등 대형 저축은행 중심 직원 수가 웰컴 저축은행 지난해 상반기 인력 명 채용 데 하반기 디지털 부문 채용 공 저축은행 올해 디지털 뱅킹 플랫폼 출범 인력 수요 페퍼 저축은행 지난해 모바일 뱅킹 앱 페퍼루 를 출시 바 저축은행 지난해 정규직 신입 경력 직원 채용 및 디지털 부문 한해 진행 신입사원 채용 및 언어 개발 가능 우대 조건 력사 대해 등 프로그램 개발 우대 한편 비대 채널 상품 기획 및 운영 경험 모바일 뱅킹 인터넷 뱅킹 기획 및 운영 경험 이 은행 최근 사업 입찰 공고 보안 간편 최고 속도 를 의미 카카오 뱅크 인터넷 전문 은행 수준 앱 구축 목표 제시 저축은행 단숨 인력 수 금융 비교 연봉 수준 한몫 분석 취업 포털 사람인 게재 정보 저축은행 대졸 초임 평균 연봉 수준 규모 경제 실현 영업 성과 직결 대형 저축은행 거래 고객 확보 위 목적 디지털 뱅킹 앱 앞 웰컴 저축은행 디지털 플랫폼 웰컴 디지털 뱅크 지난해 다운로드 수 돌파 잔돈 적금 상품 등 혁신 금융 상품 눈길 무료 사업자 매출 조회 서비스 외환 송금 체크카드 기기

### (3) SBERT 로드

In [ ]:
# 모델 로드
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

top_n = 100
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['성장', '성장 위해', '탈바꿈 고객', '한국 투자', '출시 저축은행', '은행', '저축은행 거래', '은행 최근', '웰컴 저축은행', '취업 포털', '경험 모바일', '가망 고객', '투자', '역시 디지털', '규모 경제', '증가 성장', '금융감독원', '고객 고객', '혁신', '올해 성장', '매출', '웰뱅 저축은행', '성장 추세', '금융감독원 금융', '고객 기반', '경제', '주요 고객', '조성 금융감독원', '디지털', '고객 중장', '금융 비교', '오픈 뱅킹', '상품 혁신', '인터넷', '투자 환경', '고객 대거', '사업자', '기준 저축은행', '사업', '은행 업무', '목적 디지털', '업계', '경험 은행', '시중 은행', '금융 서비스', '금융', '디지털 플랫폼', '디지털 탈바꿈', '금융 상품', '은행 수준', '금융 통계', '사업 입찰', '디지털 변화', '전문 은행', '하반기 디지털', '업계 올해', '최근 사업', '인력 금융', '뱅킹 기획', '정보 저축은행', '인터넷 전문', '저축은행 한국', '시행 저축은행', '통해 금융', '디지털 부문', '경제 실현', '저축은행 올해', '영업 성과', '저축은행 대형', '저축은행 설명', '올해 디지털', '모바일 뱅킹', '저축은행 저축은행', '뱅킹 플랫폼', '사업자 매출', '투자 저축은행', '저축은행', '무료 사업자', '뱅킹 시행', '비롯 저축은행', '저축은행 비롯', '디지털 인재', '저축은행 이용', '저축은행 가망', '저축은행 중심', '대형 저축은행', '저축은행 업계', '저축은행 개사', '저축은행 단숨', '인터넷 뱅킹', '디지털 뱅킹', '디지털 뱅크', '뱅킹 인터넷', '업계 디지털', '혁신 금융', '뱅크 인터넷', '실제 저축은행', '저축은행 대졸', '디지털 투자', '저축은행 디지털']


### (4) Max Sum Similarity

In [ ]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [ ]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=10, nr_candidates=10)

['디지털 뱅킹',
 '디지털 뱅크',
 '뱅킹 인터넷',
 '업계 디지털',
 '혁신 금융',
 '뱅크 인터넷',
 '실제 저축은행',
 '저축은행 대졸',
 '디지털 투자',
 '저축은행 디지털']

In [ ]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=10, diversity=0.5)

['저축은행 디지털',
 '고객 대거',
 '혁신 금융',
 '인터넷 전문',
 '무료 사업자',
 '저축은행 한국',
 '언어 개발',
 '뱅크 인터넷',
 '업계 올해',
 '대형 저축은행']

## [4] keyBert
+ 공식문서 
    + https://github.com/MaartenGr/KeyBERT
    + https://maartengr.github.io/KeyBERT/index.html#installation

+ 참고문서
    + https://otugi.tistory.com/m/352

In [ ]:
! pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 237 kB 12.1 MB/s 
     |████████████████████████████████| 51 kB 6.0 MB/s 


In [ ]:
from keybert import KeyBERT
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(tokenized_nouns)

In [ ]:
 kw_model.extract_keywords(tokenized_nouns, keyphrase_ngram_range=(1, 2), stop_words=None)

[('저축은행 저축은행', 0.575),
 ('저축은행 이용', 0.5086),
 ('채용 저축은행', 0.4967),
 ('저축은행 설명', 0.4905),
 ('저축은행 대형', 0.4881)]

## [5] 고전적 방법

### (1) 한국어 형태소 분석기 설치

In [6]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2022-10-24 13:58:12--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2022-10-24 13:58:12--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5ff38377a81bf17da96afd3abd.dl.dropboxusercontent.com/cd/0/get/BvaP5SZNgC2a54vi1dRgMkuiW4yl3-Oczsavf4qCXK3MdCABVGbVe05JR1qFJOIQYe4GIUzzoN4w2yZ3M-zDYYlF3LK3hq24RnXXNhW2MW1kSqfgsZlUpjxf4qxwTYeIMuYu0KTJpVmCEN1AvQGvsADMvbpRLcSiYjHMhVXFR_5258eEA_sKbSxUG79QC_gR7AE/file?dl=1# [following]
--2022-10-24 13:58:13--  https://uc5ff38377a81bf17da96afd3abd.dl.dropboxusercontent.com/cd/0/get/BvaP5SZNg

### (2) 형태소 분석을 이용한 명사 추출, 불용어
+ 아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼	어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다	하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 조 말 중 억 달 개 대 위 시 점 명 내 건 주 데 원 억 조 말 중 개 대 차 위 시 점 명 간 내 건 주 데 폭 장 전 층 선 곳 폼 달  앞 손 배 바 액 빅 호 부 톤 번 반 업 초 회 세 외 측 분 미


In [16]:
tmp = list()
for char in top_descriptions_nouns:
    if len(char) == 1:
        tmp.append(char)
tmp2 = ' '.join(tmp)
tmp2

'원 억 조 말 중 개 대 차 위 시 점 명 간 내 건 주 데 폭 장 전 층 선 곳 폼 달 앱 앞 법 손 배 바 액 빅 호 부 톤 번 반 업 초 금 회 세 외 측 분 미'

In [ ]:
stop_words = input().split()
stop_words = set(stop_words)

In [11]:
df1 = df0[df0['topic']=='경제']
print(df1.shape)

(7190, 4)


In [13]:
from konlpy.tag import Mecab
tagger = Mecab()

descriptions_nouns = []

for description in df1.description:
    for noun in tagger.nouns(description):
        if noun not in stop_words:
            descriptions_nouns.append(noun)



from collections import Counter
descriptions_nouns_counter = Counter(descriptions_nouns)
top_descriptions_nouns = dict(descriptions_nouns_counter.most_common(500))
top_descriptions_nouns

{'원': 25196,
 '금융': 22263,
 '억': 16882,
 '투자': 13133,
 '시장': 10899,
 '코로나': 10304,
 '은행': 10048,
 '기업': 9277,
 '조': 9041,
 '말': 9003,
 '중': 8968,
 '사업': 8613,
 '보험': 7624,
 '고객': 7089,
 '가능': 6855,
 '대출': 6822,
 '서비스': 6693,
 '대비': 6587,
 '지원': 6441,
 '올해': 6275,
 '펀드': 6114,
 '분기': 6026,
 '상품': 5971,
 '증가': 5830,
 '규모': 5550,
 '증권': 5421,
 '자산': 5413,
 '기준': 5360,
 '확대': 5347,
 '카드': 5316,
 '국내': 5315,
 '개': 5283,
 '경우': 5181,
 '전망': 5137,
 '업계': 5125,
 '관계자': 4896,
 '금리': 4874,
 '상황': 4573,
 '지난해': 4558,
 '관련': 4491,
 '영업': 4490,
 '예정': 4450,
 '최근': 4334,
 '지역': 4275,
 '이후': 4261,
 '달러': 4245,
 '제공': 4237,
 '상승': 4237,
 '수요': 4187,
 '계획': 4172,
 '분양': 4155,
 '기록': 4128,
 '가격': 4097,
 '주택': 4056,
 '수익': 4041,
 '실적': 4013,
 '글로벌': 3974,
 '개발': 3963,
 '지속': 3962,
 '자금': 3918,
 '그룹': 3884,
 '거래': 3855,
 '공급': 3830,
 '대': 3786,
 '진행': 3783,
 '차': 3761,
 '성장': 3675,
 '산업': 3674,
 '때문': 3620,
 '감소': 3585,
 '회사': 3552,
 '한국': 3538,
 '강화': 3531,
 '판매': 3530,
 '예상': 3517,
 '투자자': 3505,
 '개선': 